## Main Objective is To find the relationship between car prices and body style, size and engine specs & To Find Do Body Style, Size and Engine Specs predict car prices?

I had cleaned the data that was retrieved from the database and done some data cleaning and feature engineering in our previous week assignment and saved in csv file named as "cars.csv" so, for this assignment I will be uisng that csv data to train our model.

I had generated 6 features from feature engineering process and the columns are 'fuel_efficiency', 'vehicle_volume', 'engine_power_density', 'performance_index', 'engine_performance','power2weight_ratio'

I will be using 3 features in my training phase, column 'performance_index' and 'engine_performance' as mandatory columns because they had good feature imporatance score and other one will be used in random combination and will be decided which one to use after I analysed the performance of the model iteratively. I will keep track of each iteration and find the best combination of 3 features with optimal and best score of the model at the end

In [1]:
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
df = pd.read_csv('cars.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   symboling             183 non-null    int64  
 1   normalized-losses     183 non-null    float64
 2   make                  183 non-null    int64  
 3   fuel-type             183 non-null    int64  
 4   aspiration            183 non-null    int64  
 5   num-of-doors          183 non-null    int64  
 6   body-style            183 non-null    int64  
 7   drive-wheels          183 non-null    int64  
 8   engine-location       183 non-null    int64  
 9   wheel-base            183 non-null    float64
 10  length                183 non-null    float64
 11  width                 183 non-null    float64
 12  height                183 non-null    float64
 13  curb-weight           183 non-null    int64  
 14  engine-type           183 non-null    int64  
 15  num-of-cylinders      1

In [4]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,price,num_doors,num_cylinders,engine_size_cc,power2weight_ratio,vehicle_volume,engine_power_density,fuel_efficiency,performance_index,engine_performance
0,2,164.0,0,1,0,0,3,1,0,99.8,...,13950.0,4,4,1786,0.043646,634816.956,0.935780,27.0,240.051348,255.0
1,2,164.0,0,1,0,0,3,0,0,99.4,...,17450.0,4,5,2228,0.040722,636734.832,0.845588,20.0,223.973088,184.0
2,2,161.0,0,1,0,1,3,1,0,99.8,...,15250.0,2,5,2228,0.043877,624189.969,0.808824,22.0,241.324292,187.0
3,1,158.0,0,1,0,0,3,1,0,105.8,...,17710.0,4,5,2228,0.038678,766364.046,0.808824,22.0,212.728551,187.0
4,1,161.0,0,1,0,0,4,1,0,105.8,...,18920.0,4,5,2228,0.037238,766364.046,0.808824,22.0,204.807041,187.0


In [5]:
df_copy = df

In [6]:
df_with_power2weight_ratio = df_copy.drop(columns=['fuel_efficiency', 'vehicle_volume', 'engine_power_density'])

In [7]:
df_with_fuel_efficiency = df_copy.drop(columns=['vehicle_volume', 'engine_power_density', 'power2weight_ratio'])

In [8]:
df_with_vehicle_volume = df_copy.drop(columns=['fuel_efficiency', 'engine_power_density', 'power2weight_ratio'])

In [9]:
df_with_engine_power_density = df_copy.drop(columns=['fuel_efficiency', 'vehicle_volume', 'power2weight_ratio'])

In [10]:
# List of DataFrames
dfs = {
    "power2weight_ratio": df_with_power2weight_ratio,
    "fuel_efficiency": df_with_fuel_efficiency,
    "vehicle_volume": df_with_vehicle_volume,
    "engine_power_density": df_with_engine_power_density
}

In [11]:
results = {}

### I will be using Random Forest Regressor as the ML model to predict

In [12]:
for name, df in dfs.items():
    print(f"Training model on {name} dataset...")

    # Defining features (X) and target (y)
    X = df.drop(columns=["price"])
    y = df["price"]

    # Splitting into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initializing and training RandomForestRegressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Evaluating performance
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Storing results
    results[name] = {"R² Score": r2, "MAE": mae}

Training model on power2weight_ratio dataset...
Training model on fuel_efficiency dataset...
Training model on vehicle_volume dataset...
Training model on engine_power_density dataset...


In [13]:
# Sorting results by R² Score in descending order
sorted_results = sorted(results.items(), key=lambda x: x[1]["R² Score"], reverse=True)

# Printing results
print("\nModel Performance (Best to Worst):")
for dataset, metrics in sorted_results:
    print(f"\nDataset: {dataset}")
    print(f"R² Score: {metrics['R² Score']:.4f}")
    print(f"Mean Absolute Error: {metrics['MAE']:.2f}")


Model Performance (Best to Worst):

Dataset: power2weight_ratio
R² Score: 0.9335
Mean Absolute Error: 1280.55

Dataset: fuel_efficiency
R² Score: 0.9324
Mean Absolute Error: 1266.35

Dataset: engine_power_density
R² Score: 0.9310
Mean Absolute Error: 1291.10

Dataset: vehicle_volume
R² Score: 0.9289
Mean Absolute Error: 1304.05


After multiple iteration I am in concludion that engineered features like 'power2weight_ratio','performance_index' and 'engine_performance' will give me the best and optimal model. I will be using those features to train the model.

In [14]:

df = df_with_power2weight_ratio  # Ensure this dataset is preloaded

# Defining features and target variable
X = df.drop(columns=['price'])  
y = df['price']  

# Splitting the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Saving the validation set
val_set = pd.concat([X_val, y_val], axis=1)
val_set.to_csv("validation_set.csv", index=False)

# Training the RandomForest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Saving the trained model
joblib.dump(rf_model, "random_forest_model.pkl")

['random_forest_model.pkl']

The dataset is split into training (70%), validation (15%), and test (15%) sets. After training a Random Forest Regressor model, the validation set and the trained model are saved for future use.